In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image  , ImageDraw 
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os
import json
#from google.colab import drive
#drive.mount('/content/drive')


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My Drive/Resnet_Beta-Version

/content/drive/My Drive/Resnet_Beta-Version


## **Test1 Raw Hand Face Pose As Depth**


In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
       #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}
class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])
        self.transform = hm_transform
        img_path = path
        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0])
        hm_face , hm_hand = self.get_heatmap(ann_path + '.txt',img_w,img_h)  
        hm_pose = self.get_heatmap_pose(ann_path + '.json',img_w,img_h)
        
        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)
            hm_hand = hm_transform(hm_hand)
            hm_pose = torch.stack([ hm_transform(x) for x in hm_pose ], dim=1).squeeze(0)


        #print( hm_pose.size() )
        image = torch.cat((image,hm_face,hm_hand,hm_pose))
       
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():
                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1

    def get_heatmap_pose(self,ann_path, img_w, img_h):
        
        heatmaps = [ Image.new('L', (img_w, img_h)) for x in range(17) ]

        try:
            with open(ann_path,'r') as f:
                data = json.load(f)
            #print(ann_path)
            #print(data)  
            try:
                keypoints = data[0]['keypoints']
            except IndexError:
                return heatmaps

            #print(keypoints)
            # TODO check for two key mappings -- working

            for i in range(0,len(keypoints),3):

                xmin = keypoints[i] - 5
                ymin = keypoints[i+1] - 5
                xmax = keypoints[i] + 5
                ymax = keypoints[i+1] + 5

                hmDraw = ImageDraw.Draw(heatmaps[i//3]) 
                hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")

        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return heatmaps



input_path = './alpha-dataset'

image_datasets = {
    # 'train': 
    # MyImageFolder(input_path + '/train_m', data_transforms['train']),
    #  'validation': 
    # MyImageFolder(input_path + '/val_m', data_transforms['validation']),
    'test':
    MyImageFolder(input_path + '/test_m', data_transforms['test']),
    
}

dataloaders = {
    # 'train':
    # torch.utils.data.DataLoader(image_datasets['train'],
    #                             batch_size=64,
    #                             shuffle=True,
    #                             num_workers=8),  # for Kaggle
    # 'validation':
    # torch.utils.data.DataLoader(image_datasets['validation'],
    #                             batch_size=16,
    #                             shuffle=True,
    #                             num_workers=8) , # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=8)  # for Kaggle
}

In [0]:
epoch = 65
device = 'cuda'
model = torch.load('epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    print("iteration"+str(iteration))


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torchvision.models.resnet.ResNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torchvision.models.resnet.Bottleneck' has changed. you can retrieve the original source code by accessing the object's source attribute

iteration1
iteration2
iteration3
iteration4
iteration5
iteration6
iteration7
iteration8
iteration9
iteration10
iteration11
iteration12
iteration13
iteration14
iteration15
iteration16
iteration17
iteration18
iteration19
iteration20
iteration21
iteration22
iteration23
iteration24
iteration25
iteration26
iteration27
iteration28
iteration29
iteration30
iteration31
iteration32
iteration33
iteration34
iteration35
iteration36
iteration37
iteration38
iteration39
iteration40
iteration41
iteration42
iteration43
iteration44
iteration45
iteration46
iteration47
iteration48
iteration49
iteration50
iteration51
iteration52
iteration53
iteration54
iteration55
iteration56
iteration57
iteration58
iteration59
iteration60
iteration61
iteration62
iteration63
iteration64
iteration65
iteration66
iteration67
iteration68
iteration69
iteration70
iteration71
iteration72
iteration73
iteration74
iteration75
iteration76
iteration77
iteration78
iteration79
iteration80
iteration81
iteration82
iteration83
iteration84
i

In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 487
total 553
0.8806509945750453


## TEST 2 (Raw + Hand)

In [0]:


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
       #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}
class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])



        img_path = path
        #print("--i: %d , TA: %s"%(i,target_acquired))

        
        #img_path = target_acquired
        #label =  target_acquired[1]

        #print(img_path)

        image = Image.open(img_path)
     
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        #image = np.dstack((image, hm_face))

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_hand = hm_transform(hm_hand)

        
        #imgplot = plt.imshow(image)
        #plt.show()
        image = torch.cat((image,hm_hand))
        
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():

                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1
input_path = './alpha-dataset'

image_datasets = {
    # 'train': 
    # MyImageFolder(input_path + '/train_m', data_transforms['train']),
    #  'validation': 
    # MyImageFolder(input_path + '/val_m', data_transforms['validation']),
    'test':
    MyImageFolder(input_path + '/test_m', data_transforms['test']),
    
}

dataloaders = {
    # 'train':
    # torch.utils.data.DataLoader(image_datasets['train'],
    #                             batch_size=64,
    #                             shuffle=True,
    #                             num_workers=8),  # for Kaggle
    # 'validation':
    # torch.utils.data.DataLoader(image_datasets['validation'],
    #                             batch_size=16,
    #                             shuffle=True,
    #                             num_workers=8) , # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=8)  # for Kaggle
}

In [0]:
image_datasets['test'].classes


['Falling', 'NonFalling']

In [0]:

epoch = 66
device = 'cuda'
model = torch.load('weights/alpha/raw-hand/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)
    #imgplot = plt.imshow(inputs)
  
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        #print ("pred_logits_tensorFalling "+ str(pred_logits_tensorFalling))
        #print ("LABELS :" +str(labels))
        target.append(labels)

    #print("iteration"+str(iteration))


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 
print (out)
acc = 0
for i,tar in enumerate(target):
    #print (tar) 
    #print ("["+str (tar.item())+","+ str (out[i])+"]")
    if tar.item() == out[i]:
      
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

[1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 

## Test 3 .Raw Face Hand

In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
       #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):


        path, target = self.samples[index]
       

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])

      

        img_path = path
        

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)
            hm_hand = hm_transform(hm_hand)

        image = torch.cat((image,hm_face,hm_hand))
       
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():
                    
                    if line == '\n':
                        continue

                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1
input_path = './alpha-dataset'

image_datasets = {
    # 'train': 
    # MyImageFolder(input_path + '/train_m', data_transforms['train']),
    #  'validation': 
    # MyImageFolder(input_path + '/val_m', data_transforms['validation']),
    'test':
    MyImageFolder(input_path + '/test_m', data_transforms['test']),
    
}

dataloaders = {
    # 'train':
    # torch.utils.data.DataLoader(image_datasets['train'],
    #                             batch_size=64,
    #                             shuffle=True,
    #                             num_workers=8),  # for Kaggle
    # 'validation':
    # torch.utils.data.DataLoader(image_datasets['validation'],
    #                             batch_size=16,
    #                             shuffle=True,
    #                             num_workers=8) , # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=8)  # for Kaggle
}

In [0]:

epoch = 13
device = 'cuda'
model = torch.load('weights/alpha/raw-face-hand/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    #print("iteration"+str(iteration))


iteration1
iteration2
iteration3
iteration4
iteration5
iteration6
iteration7
iteration8
iteration9
iteration10
iteration11
iteration12
iteration13
iteration14
iteration15
iteration16
iteration17
iteration18
iteration19
iteration20
iteration21
iteration22
iteration23
iteration24
iteration25
iteration26
iteration27
iteration28
iteration29
iteration30
iteration31
iteration32
iteration33
iteration34
iteration35
iteration36
iteration37
iteration38
iteration39
iteration40
iteration41
iteration42
iteration43
iteration44
iteration45
iteration46
iteration47
iteration48
iteration49
iteration50
iteration51
iteration52
iteration53
iteration54
iteration55
iteration56
iteration57
iteration58
iteration59
iteration60
iteration61
iteration62
iteration63
iteration64
iteration65
iteration66
iteration67
iteration68
iteration69
iteration70
iteration71
iteration72
iteration73
iteration74
iteration75
iteration76
iteration77
iteration78
iteration79
iteration80
iteration81
iteration82
iteration83
iteration84
i

In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 462
total 553
0.8354430379746836


#  Test 4 (Raw + Face )

In [0]:

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform

    def __getitem__(self, index):

        

        path, target = self.samples[index]
       

        hm_transform = transforms.Compose([
                          transforms.Resize((224,224)),
                          transforms.ToTensor()

                                ])


        img_path = path
      

        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
        ann_path = img_path.replace(filename, filename.split('.')[0] + '.txt')
        hm_face , hm_hand = self.get_heatmap(ann_path,img_w,img_h)  

        if self.transform is not None:
            
            image = self.transform(image)
            hm_face = hm_transform(hm_face)

        image = torch.cat((image,hm_face))
        
        return image, target

    def get_heatmap(self,ann_path, img_w , img_h):
        
        hm_0 = Image.new('L', (img_w, img_h))
        hm_1 = Image.new('L', (img_w, img_h))
        
        try:

            with open(ann_path) as f:

                for line in f.readlines():
                    if (line == '\n'):
                      continue
                    line = line.split(' ')

                    cx = float(line[1])
                    cy = float(line[2])
                    w = float(line[3])
                    h = float(line[4])

                    xmin =  (cx - (w/2)) * img_w
                    ymin = (cy - (h/2)) * img_h
                    xmax = (cx + (w/2)) * img_w
                    ymax = (cy + (h/2)) * img_h

                    if line[0] == '0':
                        hmDraw = ImageDraw.Draw(hm_0)   
                    else:
                        hmDraw = ImageDraw.Draw(hm_1)   
                    
                    hmDraw.rectangle([xmin,ymin,xmax,ymax], fill ="#ffffff")
        except FileNotFoundError:
            #print("FileNotFoundError: %s"%ann_path)
            pass

        return hm_0, hm_1
input_path = './alpha-dataset'

image_datasets = {
#     'train': 
#     MyImageFolder(input_path + '/train_m', data_transforms['train']),
#      'validation': 
#     MyImageFolder(input_path + '/val_m', data_transforms['validation']),
    'test':
    MyImageFolder(input_path + '/test_m', data_transforms['test']),
    
}

dataloaders = {
    # 'train':
    # torch.utils.data.DataLoader(image_datasets['train'],
    #                             batch_size=64,
    #                             shuffle=True,
    #                             num_workers=8),  # for Kaggle
    # 'validation':
    # torch.utils.data.DataLoader(image_datasets['validation'],
    #                             batch_size=16,
    #                             shuffle=True,
    #                             num_workers=8) , # for Kaggle
   'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=8)  # for Kaggle
}

In [0]:

epoch = 62
device = 'cuda'
model = torch.load('weights/alpha/raw-face/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    #print("iteration"+str(iteration))


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 488
total 553
0.8824593128390597


# Test 5 (RAW)

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
    'test':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        #normalize
    ]),
}


input_path = './alpha-dataset'

image_datasets = {
#     'train': 
#     MyImageFolder(input_path + '/train_m', data_transforms['train']),
#      'validation': 
#     MyImageFolder(input_path + '/val_m', data_transforms['validation']),
    'test':
    datasets.ImageFolder(input_path + '/test_m', data_transforms['test']),
    
}

dataloaders = {
    # 'train':
    # torch.utils.data.DataLoader(image_datasets['train'],
    #                             batch_size=64,
    #                             shuffle=True,
    #                             num_workers=8),  # for Kaggle
    # 'validation':
    # torch.utils.data.DataLoader(image_datasets['validation'],
    #                             batch_size=16,
    #                             shuffle=True,
    #                             num_workers=8) , # for Kaggle
    'test':
    torch.utils.data.DataLoader(image_datasets['test'],
                                batch_size=1,
                                shuffle=True,
                                num_workers=8)  # for Kaggle
}

In [0]:
dataloaders['test'].dataset.classes

['Falling', 'NonFalling']

In [0]:

epoch = 21
device = 'cuda'
model = torch.load('weights/alpha/raw/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

for inputs , labels in dataloaders['test']:

    #if iteration == 150:
    #  break
    iteration += 1
    inputs = inputs.to(device)
    labels = labels.to(device)

    
    with torch.set_grad_enabled(False):
        pred_logits_tensorFalling = model(inputs)
        predictions.append(pred_logits_tensorFalling)
        target.append(labels)

    #print("iteration"+str(iteration))


In [0]:
out = []

for pred in predictions:

    out.append( torch.argmax(pred).item()) 

acc = 0
for i,tar in enumerate(target):
    if tar.item() == out[i]:
      acc += 1

print("acc", acc)
print("total",len(out))
print(acc/len(out))

acc 483
total 553
0.8734177215189873


In [0]:
epoch = 45
device = 'cuda'
model = torch.load('weights/alpha/raw/epoch_%d.pth'%(epoch)).to(device)
optimizer = optim.Adam(model.fc.parameters(),lr=0.01)
model.eval()
predictions = []
target = []

iteration = 0

transform = transforms.Compose([
    transforms.ToTensor()
])

path = './alpha-dataset/Demo/'
for i in range(1,5):

    image = Image.open(path + str(i)+'.png') 
    image = image.convert("RGB")
    image  = image.resize((224,224))
    image = np.array(image)
    image = image[...,:3]
    image = transform(image)
    print(image.size())
    #image = image[1:4]
    image = image.unsqueeze(0)
    image = image.to(device)

    with torch.set_grad_enabled(False):
        
        out = model(image)
        print(i , out)
        print('------------------------------------')





torch.Size([3, 224, 224])
1 tensor([[1.8319e-09, 1.0000e+00]], device='cuda:0')
------------------------------------
torch.Size([3, 224, 224])
2 tensor([[9.9997e-01, 8.3400e-06]], device='cuda:0')
------------------------------------
torch.Size([3, 224, 224])
3 tensor([[0.0575, 0.8292]], device='cuda:0')
------------------------------------
torch.Size([3, 224, 224])
4 tensor([[3.5269e-06, 9.9999e-01]], device='cuda:0')
------------------------------------
